In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/all4_1x5/dataset/'

#Number of files
numberOfFiles = 2555

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile, wrap=True):
    #print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    
    #print('{} x {}'.format(numberOfRows,numberOfCols))
    if(wrap==True):
        maxY = 1000
    else:
        maxY = numberOfCols-1
    dataX = np.zeros([numberOfRows,maxY],np.int8)
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(', ')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataX[rowCounter][(int(row[i])-1)%maxY]=1 + ((int(row[i])-1)//maxY)
                #if((1 + ((int(row[i])-1)//maxY))>1):
                #    print("{} data[{}][{}] = {}".format(int(row[i])-1, rowCounter,(int(row[i])-1)%maxY,1 + ((int(row[i])-1)//maxY)))
            rowCounter+=1
        f.close()
        
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    #del dataX['id']
    del dataY['id']

    return dataX, dataY.as_matrix()

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1231.csv', 'datasetY_1231.csv',wrap=True)

In [5]:
print(dataX.shape)
print(dataX[0:5])

(1241, 1000)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [6]:
print(dataY.shape)
print(dataY[0:5])

(1241, 539)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 1000
Output Classes: 539


In [20]:
# create model
# logisticRegression = Sequential(name='Simple Logistic Regression')
# logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid', init='glorot_uniform'))

# simpleModel = Sequential(name='2 Fully Connected Layers')
# simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
# simpleModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='tanh', init='glorot_uniform'))
deepModel.add(Dense(2048, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='tanh', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

models = [deepModel]

In [21]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [22]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [23]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [28]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 15
for eon in range(0, numberOfEons):
    print('Eon {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        for model in models:
            #print('Model = {}'.format(model.name))
            #model.fit(dataX, dataY, nb_epoch=5, verbose=0, batch_size=512)
            loss, f1score=fit_data(model,dataX, dataY)
            print('Learning for file {} / {} : datasetX/Y_{}\t\tloss={:.4f} f1score={:.4f}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex], loss, f1score), end='\r')
        
#             if((trainIndex+1)%10==0):
#                 sc=model.test_on_batch(dataX,dataY)
#                 print('Learning for file {} / {} : datasetX/Y_{}...'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
#                 print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, sc[0], sc[1]))
#                 print('==========================================')
             
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[testIndex]), end='\r')
        for model in models:
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    
    
    print('                                                                    ')
    print('Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))


***********************************
 1/15
==========================================_20		loss=0.0096 f1score=0.730357

==========================================8253

1.  1/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7228	 precision = 0.8599 	 recall = 0.6234


***********************************
 2/15
==========================================_20		loss=0.0086 f1score=0.759035

==========================================8253

2.  2/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7477	 precision = 0.8585 	 recall = 0.6624


***********************************
 3/15
==========================================_20		loss=0.0082 f1score=0.772842

==========================================8253

3.  3/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7589	 precision = 0.8639 	 recall = 0.6767


***********************************
 4/15
==========================================_20		loss=0.0079 f1score=0.778952

====

==================================================
# all4_1x5
==================================================
1.  1/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7228	 precision = 0.8599 	 recall = 0.6234
2.  2/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7477	 precision = 0.8585 	 recall = 0.6624
3.  3/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7589	 precision = 0.8639 	 recall = 0.6767
4.  4/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7665	 precision = 0.8661 	 recall = 0.6874
5.  5/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7733	 precision = 0.8619 	 recall = 0.7012
6.  6/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7773	 precision = 0.8675 	 recall = 0.7041
7.  7/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7808	 precision = 0.8655 	 recall = 0.7114
8.  8/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7831	 precision = 0.8697 	 recall = 0.7123
9.  9/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7852	 precision = 0.8667 	 recall = 0.7178
10.  10/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7869	 precision = 0.8693 	 recall = 0.7189
11.  11/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7884	 precision = 0.8717 	 recall = 0.7197
12.  12/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7894	 precision = 0.8702 	 recall = 0.7224
13.  13/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7908	 precision = 0.8710 	 recall = 0.7241
14.  14/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7908	 precision = 0.8710 	 recall = 0.7242
15.  15/15 - Testing score:
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7909	 precision = 0.8705 	 recall = 0.7246

## 